Vitber prosjekt 2: Maskinlæring

# Bibliotekimportering

In [1]:
#Importerer nyttige biblioteker og hjelpefunksjoner
import numpy as np
import matplotlib.pyplot as plt
from numba import jit
from plotting import plot_progression
from numba import njit
from spirals import get_data_spiral_2d
import random
import time
from loader import get_dataset
#--------------------------------------------------------------------------------------
#Definerer noen konstanter som ikke forandrer seg i løpet av gjennomkjøringene
#W er en Kxdxd-matrise (liste med W_k elementer med lengde K)
#W_k er en dxd matrise i lag k kalt en vekter   
#b_k er en d-vektor kalt biaset til lag k       
#Y er en dxI matrise med alle bildene. I er antall bilder
#omega er en d-vektor som er en parameter
#my er en skalar som er en parameter
#U er en informasjonsvektor med [[[W_0,b_0],[W_1,b_1],...[W_K-1,b_K-1]],omega,my]. K er antall lag
#Z er prosjisering
#c er fasiten
h = 0.1 #skrittlengde
###Foreslår å definere d og I, og bruke dem videre. Blir mer oversiktlig for oss selv med tanke på hva som er hva slags type matrise og sånt
#--------------------------------------------------------------------------------------


# Diverse tekster det er lurt å lese

**Forslag**:


1.   Foreslår å definere d og I, og bruke dem videre. Blir mer oversiktlig for oss selv med tanke på hva som er hva slags type matrise og sånt
2.   Foreslår å bytte ut .dot() med @ som betyr akkurat det samme men er enklere å lese. Gjort
3. Droppe egt hele U-funksjongreia. Gjort
4. Rename algoritme til treningsfunk eller lignende. Kanskje ha W,Y,C osv som inputs i den funksjonen?
5. Utføre oppgaven til P_finder i algoritmen sånna t vi slipper å lage yliste
6. implementere Jit




**Spørsmål**:


1.   Hvordan laster jeg ned 2d_spirals på en måte at den kan brukes i dette programmet?
2.   List item


**Svar**:


1.   På venstre side står det en filfolder der du må laste opp filene for hver gang man lager en runtime (typ hver gang vi starter å jobbe)
2.   List item




**Endringslogg**:


03.03 ettermiddag Sindre
1.   Fikset noen småfeil i rekkefølgen på .dot() i en del derivert-funksjoner.
2.   Laget **P_finder** som lager [0-matrise, P_1, P_2,..., P_K]
3.   Skrevet noen forslag
4.   Omstrukturert
5.   Sett over mange av funksjonene. Godt jobba gutta:3

04.03 formiddag Markus, Jostein og Sindre
1. Svart på spørsmål 1
2. Byttet ut .dot med @
3. Debugga/endra ting for å få ting til å kjøre
4. Fjernet U
5. Laget makeW og makeb

05.03 formiddag Markus, Jostein
1. Fikset dJdmy, den skulle returnere skalar, men returnerte liste...
2. laget answercheck(Z,C) som returnerer %riktig

# Enkle hjelpefunksjoner

In [37]:
#definerer noen små hjelpefunksjoner som brukes.


def objektfunksjon(Z, C):#Returnerer J, som er en skalar
    J= 0.5 * sum((Z-C)**2) #Bruker at numpy er genialt
    return J

@njit
def sigma(X): #Sigmoid function
    #Hvis Mnist:
    #-----------
    #X = X/1000
    #-----------
    return np.tanh(X/1000)

@njit
def sigmader(X):#Deriverte av sigma
    return -sigma(X)**2+1
    
@njit
def etta(X): #Aktiveringsfunk
    return 0.5*(1+sigma(X/2))

@njit
def ettader(X): #Deriverte av etta
    return (1/4)*(-(sigma(X/2))**2+1)

@njit
def makeW(K,d): #Lager W som en Kxdxd-matrise hvor alle elementene er mellom 0 og 1
    return np.random.rand(K,d,d)
    

def makeb(K,d,I): #Lager b som en kxdxI-matrise
    standard= np.random.uniform(0, 1, (K,d) )
    standard=list(standard)
    for j in range(K):
        standard[j]=list(standard[j])
    for j in range(K):
        standard[j]=[standard[j]]
        for i in range(I-1):
            standard[j].append(standard[j][i])
        standard[j]=np.array(standard[j]).transpose()
    return np.array(standard)

#Gjør om en batchet B til en med riktig størrelse. (shortB=KxdxI_batch, output=KxdxI_tot)
def finalB(shortB,I_tot): 
    d=len(shortB[0])
    K=len(shortB)
    
    standard= np.zeros((K,d))
    standard=list(standard)
    #Gjør standard til en Kxd med riktige verdier
    for k in range(K):
        for i in range(d):
            standard[k][i]=shortB[k][i][0]
    
    #Gjør standard til KxdxI_max
    for j in range(K):
        standard[j]=list(standard[j])
    for j in range(K):
        standard[j]=[standard[j]]
        for i in range(I_tot-1):
            standard[j].append(standard[j][i])
        standard[j]=np.array(standard[j]).transpose()
    return np.array(standard)

def answercheck(Z,C):#Sjekker verdier og fasit og returnerer hvor stor prosentandel er rikitg
    correct=0
    total=0
    for i in range(len(Z)):
        if round(Z[i],0)==C[i]:
            correct+=1
            total+=1
        else:
            total+=1
    return correct*100/total

def formatC(C): #Omdanner fasit C til noe lettere. Brukes på spirals
    nyC=[0 for i in range(len(C))]
    for i in range(len(C)):
        if C[i]:
            nyC[i]+=(1)
    return np.array(nyC)

def zconvert(Z):#Gjør ingenting lol
    for el in Z:
        el=round(el,0)
    return Z

def makeOmegaMyWb(K,d,I):#Initialiserer W,b,omega og my
    omega= np.random.rand(d) /d #definerer omega som d-vektor med tall i [0,1/d]
    my=random.random() /d       #definerer my som tall i [0,1/d]
    W=makeW(K,d) /d             #definerer W som Kxdxd med tall i [0,1/d]
    b=makeb(K,d,I) /d           #definerer b som KxdxI med tall i [0,1/d]
    return omega, my, W, b

# Derivertfunksjoner

In [3]:
#------------------------------------------------------------------------------------
@njit
def dJdmy(Yk, omega, my, Z, C, ones):
    return ettader( Yk.transpose()@omega + my*ones ).transpose()@(Z-C)

#@njit
def dJdomega(Yk,omega, my, Z, C, ones):
    return Yk @ ( (Z - C) * ettader(Yk.transpose()@omega + my*ones ) )#my må kanskje ganges med enhetsmatrisen. My skal vel være dx1-matrise?

#@njit
def dJdW(P_kplus1, W_k, Y_k, b_k):
    return h * (P_kplus1 * sigmader(W_k@Y_k + b_k))@Y_k.transpose()

#@njit
def dJdb(P_kplus1, W_k, Y_k, b_k,ones):
    return h*(P_kplus1 * sigmader(W_k@Y_k + b_k))@ones #Lengden er riktig da Y_k er dx

#P-funksjoner


In [4]:
#Funksjoner for back-propagation

def P_K(Y_K, omega, my, Z, C):
    I=len(Y_K[0])

    #Kreativ måte å få dx1 @ 1xI = dxI 
    faktor1=omega
    faktor1=np.array([faktor1 for i in range(len(faktor1))]).transpose()
    faktor2=( (Z-C) * ettader(Y_K.transpose()@omega + my*np.ones(I)) ).transpose()
    faktor2=np.array([faktor2 for i in range(len(faktor1))])
    return (faktor1@faktor2)/2


def P_kmin1(P_k,W_kmin1,Y_kmin1, b_kmin1): 
    return P_k + h*W_kmin1.transpose()@(sigmader(W_kmin1@Y_kmin1 + b_kmin1) * P_k)

def P_finder(Y,W,b,omega,my,Z,C,zeros): #Returnerer array [0-matrise, P_1, P_2, ..., P_K]
    K=len(W); I=len(Y[0][0]); d=len(W[0])
    P=zeros #zeros er (K+1)xdxI, og brukes kun her! altså bare positivt om bruker reference. De gamle verdiene har null å si

    P[-1]=P_K(Y[-1],omega,my,Z,C)#Legger til siste element, P_K
    for k in range(K,1,-1): #Representerer at vi regner P_k. (Går fra K-1 siden K allerede er funnet, og til og med 1, siden P_1 er den laveste vi trenger)
        P[k-1] = P_kmin1(P[k] , W[k-1], Y[k-1], b[k-1]) #Legger til den nye P_k i starten av P-lista

    return P


#Mer komplekse hjelpefunksjoner

In [29]:
#------------------------------------------------------------------------------------
#Flere hjelpefunksjoner, men disse er litt mer diffuse:

#Anvender vektene og biasene på bildene of transformerer dem
def bildetransformasjon(Yk, Wk, bk): #transformerer bildevektorene
    return Yk + h*sigma(Wk@Yk+bk)

#Prosjiserer bildematrisen og spytter ut en Z-vektor
def prosjisering(Y_K,omega,my): 
    return etta( Y_K.transpose()@omega+my*np.ones(len(Y_K[0])) ) #Merk Y[0], eller Y i len()???
  
#Lager en liste med Y etter transformasjon av W,b (Altså [Y_0, Y_1,..., Y_(K-1)])
def Yfinder(W,b,Y):
    Ylist=[0 for i in range(len(W))]
    Ylist[0]=Y
    for i in range(1,len(W)):
        Y=bildetransformasjon(Y,W[i],b[i])
        Ylist[i]=Y
    return Ylist  
#----------------------------------------------------------------------------------------------------
#Nablaoperator
def nabJ(W, b , omega, my,Y,C, dJdWNy, dJdbNy,ones,zeros):
    K=len(W)
    Y_Kmin1 = Y[-1] #Dette er da det siste elementet i Y (som er [Y_0, Y_1,..., Y_(K-1)])

    #Finner Z
    Z = prosjisering(Y_Kmin1, omega, my)
    #Finner dJdomega=dx1 og dJdmy=skalar
    dJdomegaNy = dJdomega(Y_Kmin1, omega, my, Z, C, ones[0])
    dJdmyNy = dJdmy(Y_Kmin1, omega, my, Z, C, ones[0])

    #Finner P ([P_0, P_1,..., P_K], hvor P_0 er nullmatrise)
    P = P_finder(Y,W,b,omega,my,Z,C,zeros)
    #Iterer fra [0,K-1] og finner gjennom hver iterasjon dJdW_k og dJdb_k, som til sammen blir hele dJdW=Kxdxd og dJdb=KxdxI
    for i in range(0, K):
        dJdWNy[i]=dJdW(P[i+1], W[i], Y[i], b[i])
        dJdbNy[i]=dJdb(P[i+1],W[i],Y[i],b[i],ones)

    return dJdWNy,dJdbNy,dJdomegaNy,dJdmyNy
#------------------------------------------------------------------------------------------------------
#Vanilla-optimering
def optimeringsfunk(W, b , omega, my,Y,C,tau, dJdWNy, dJdbNy,ones, zeros): #Beveger seg tau steg i retning som minimerer J mest (altså -nabJ)
    W2,b2,omega2,my2 = nabJ(W, b , omega, my, Y, C, dJdWNy, dJdbNy,ones, zeros) #Henter nabJ
    W-= tau*W2;  b-= tau*b2;  omega-= tau*omega2;  my-= tau*my2
    return W,b,omega,my

#------------------------------------------------------------------------------------------------



# Adam decent optimering

In [17]:
#-----------------------------------------------------------------------------------------------
#Adam-descent-algortihm
beta1 = 0.9
beta2 = 0.999
alpha = 0.1
epsilon = 10**(-8)

def Adam(W,b, omega, my, Y, C, dJdWNy,dJdbNy, ones,zeros,m0W,m0b,m0omega, m0my, v0W,v0b, v0omega,v0my, i):

    #Finner g_j
    dJdW, dJdb, dJdomega, dJdmy = nabJ(W, b , omega, my,Y,C, dJdWNy, dJdbNy, ones,zeros)
    
    #m_j
    m0W = beta1*m0W + (1-beta1)*dJdW
    m0b = beta1*m0b + (1-beta1)*dJdb
    m0omega = beta1*m0omega + (1-beta1)*dJdomega
    m0my = beta1*m0my + (1-beta1)*dJdmy
    
    #v_j
    v0W = beta2*v0W + (1-beta2)*(dJdW*dJdW)
    v0b = beta2*v0b + (1-beta2)*(dJdb*dJdb)
    v0omega = beta2*v0omega + (1-beta2)*(dJdomega*dJdomega)
    v0my = beta2*v0my + (1-beta2)*(dJdmy*dJdmy)
    
    #mhatt_j
    mhattW = m0W / ( 1 - beta1**(i+1) ) #i+1 siden vi starter indekser på 0, mens denne skal starte på 1
    mhattb = m0b / ( 1 - beta1**(i+1) )
    mhattomega = m0omega / ( 1 - beta1**(i+1) )
    mhattmy = m0my / ( 1 - beta1**(i+1) )
    
    #vhatt_j
    vhattW = v0W / ( 1 - beta2**(i+1) )
    vhattb = v0b / ( 1 - beta2**(i+1) )
    vhattomega = v0omega / ( 1 - beta2**(i+1) )
    vhattmy = v0my / ( 1 - beta2**(i+1) )
    
    #U_(j+1)
    W -= alpha * mhattW / ( np.sqrt(vhattW) + epsilon )
    b -= alpha * mhattb / ( np.sqrt(vhattb) + epsilon )
    omega -= alpha * mhattomega / ( np.sqrt(vhattomega) + epsilon )
    my -= alpha * mhattmy / ( np.sqrt(vhattmy) + epsilon )
  
    return W, b, omega, my, m0W, m0b, m0omega, m0my, v0W, v0b, v0omega, v0my
    

# Generell treningsalgoritme

In [41]:
#Initialiserer og trener W,b,omega,my på et treningssett maxiter ganger, eller når toleransen er nådd
def generalTrain(pictures,fasit,K,tol):
    starttidspunkt=time.time()

    tau=0.1
    I=len(pictures[0]); d=len(pictures)

    #Henter ut noen startverdier basert på bildeformat
    omega,my,W,b = makeOmegaMyWb(K,d,I)  #omega=dx1, my=skalar, W=Kxdxd, b=KxdxI,    tall innhold i [0,1/d]

    #Definerer noen matriser for bruk i senere funksjoner
    Kdd0 = np.zeros((K, d,d)) #Kxdxd med 0 i hvert element
    KdI0 = np.zeros((K, d, I)) #KxdxI med 0 i hvert element
    II1=np.ones((I,I)) #IxI med 1 i hvert element
    zeros=np.zeros((K+1,d,I))#Brukes i P-finder

    #---
    iter=0 
    maxiter=1000 #Øvre iterasjonsgrense
    accuracy=0 #Starter med 0% riktige
    #---

    #Lager en matrise lik W men med 0 i alle elementer tilsvarende for b,omega og my
    m0W=0*np.copy(W); m0b=0*np.copy(b); m0omega=0*np.copy(omega); m0my=0*np.copy(my)
    v0W=np.copy(m0W); v0b=np.copy(m0b); v0omega=np.copy(m0omega); v0my=np.copy(m0my)

    print("Initsialisering ferdig...")

    #
    while(accuracy<tol and iter<maxiter):#Begynner å iterere

        Y=Yfinder(W,b,pictures) #Applyer W og b til bildet
        Z=prosjisering(Y[-1],omega,my) #Får ut liste med sannsynligheter

        #---
        #Regner ut og oppdaterer W,b,omega,my etter gradienten
        #Adam
        W,b,omega,my, m0W, m0b, m0omega, m0my, v0W, v0b, v0omega, v0my = Adam(W,b, omega, my, Y, fasit, Kdd0,KdI0, II1,zeros, m0W,m0b,m0omega, m0my, v0W,v0b, v0omega,v0my, iter)
        #Vanilla
        #W,b,omega,my=optimeringsfunk(W,b,omega,my,Y,fasit,tau, Kdd0, KdI0,II1,zeros)
        #---
        print("avvik:",objektfunksjon(Z,fasit))
        iter+=1
        print(answercheck(Z,fasit))
        if iter%10==0: #Skriver ut accuracy etter 10 iterasjoner
            accuracy=answercheck(Z,fasit)
            print("Iterasjon:",iter,"føreløpig treffsikkerhet:",answercheck(Z,fasit),"%")

    sluttidspunkt=time.time()
    print("Kjøretid:",sluttidspunkt-starttidspunkt,"\nTreffsikkerthet:",answercheck(Z,fasit),"%")

    return W,b,omega,my,answercheck(Z,fasit) #returnerer opptrente parametre samt siste accuracy-måling
#---------------------------------------------------------------------------------


#---------------------------------------------------------------------------------------
#Trener W,b,omega,my på et treningssett maxiter ganger, eller når toleransen er nådd
def trainAgain(W,b,omega,my,Y0,C,K,tol):
    starttidspunkt=time.time()

    I=len(Y0[0]); d=len(Y0)
    tau=0.1

    #Definerer noen matriser for bruk i senere funksjoner
    Kdd0 = np.zeros((K, d,d)) #Kxdxd med 0 i hvert element
    KdI0 = np.zeros((K, d, I)) #KxdxI med 0 i hvert element
    II1=np.ones((I,I)) #IxI med 1 i hvert element
    zeros=np.zeros((K+1,d,I))#Brukes i P-finder
    #---
    iter=0 
    maxiter=1000 #Øvre iterasjonsgrense
    accuracy=0 #Starter med 0% riktige
    #---
    #Lager en matrise lik W men med 0 i alle elementer tilsvarende for b,omega og my
    m0W=0*np.copy(W); m0b=0*np.copy(b); m0omega=0*np.copy(omega); m0my=0*np.copy(my)
    v0W=np.copy(m0W); v0b=np.copy(m0b); v0omega=np.copy(m0omega); v0my=np.copy(m0my)

    acchistory=[]
    while(accuracy<tol and iter<maxiter):#Begynner å iterere

        Y=Yfinder(W,b,Y0) #Applyer W og b til bildet
        Z=prosjisering(Y[-1],omega,my) #Får ut liste med sannsynligheter
    
        #---
        #Regner ut og oppdaterer W,b,omega,my etter gradienten
        #Adam
        W,b,omega,my, m0W, m0b, m0omega, m0my, v0W, v0b, v0omega, v0my = Adam(W,b, omega, my, Y, C, Kdd0,KdI0, II1,zeros, m0W,m0b,m0omega, m0my, v0W,v0b, v0omega,v0my, iter)
        #Vanilla
        #W,b,omega,my=optimeringsfunk(W,b,omega,my,Y,C,tau, Kdd0, KdI0,II1,zeros)
        #---

        
        iter+=1
        accuracy=answercheck(Z,C)
        acchistory.append(accuracy)
        print(acchistory[-1])
        print("Avvik: ",objektfunksjon(Z,C))
        if iter%10==0:
            print("Iterasjon:",iter,"føreløpig treffsikkerhet:",acchistory[-1],"%")

    sluttidspunkt=time.time()
    print("Kjøretid:",sluttidspunkt-starttidspunkt,"\nTreffsikkerthet:",acchistory[-1],"%")

    return W,b,omega,my,acchistory[-1]

  

# Generell Trening med batching

In [8]:
#Funksjon som deler opp bildene i pakker og kjører pakkevis

def batchTrain(pictures,fasit,K,tol,batches):
    #---
    #Lager currentbatch og currentfasit
    batchsize=len(pictures[0])//batches#Gir antall bilder per batch
    picinvert=pictures.transpose() #inverterer bilde-matrisen
    currentbatch=picinvert[0:batchsize].transpose() #Lager et mindre bilde med lengde batchsize
    currentfasit=fasit[0:batchsize]
    #---
    #Bruker generalTrain første gangen for å initialisere W,b,omega,my
    W,b,omega,my,currentacc=generalTrain(currentbatch,currentfasit,K,tol)#Initsialiserer W,b,omega og my og trener dem på batch 1
    
    #---
    #Trener samme W,b,omega og my på nye bildebatcher
    for i in range(1,batches-1):
        #Henter ny currentbatch og currentfasit
        currentbatch=picinvert[batchsize*i:batchsize*(i+1)].transpose()
        currentfasit=fasit[batchsize*i:batchsize*(i+1)]
        #Trener W,b,omega,my
        W,b,omega,my,currentacc=trainAgain(W,b,omega,my,currentbatch,currentfasit,K,tol)

        #Dersom acc akkurat nå er god nok for tol, så breaker den ut av forløkka og returnerer W,b,omega,my
        if currentacc>tol:
            print("Breaker fordi god nok toleranse")
            break

    print("Accuracy fra siste batchen:",currentacc)
    return W,b,omega,my #MERK!! Tar ikke med restbilder om bilder%batches >0(dvs rest)


#Y0,C=get_data_spiral_2d(5000)
#C=formatC(C)
#W,b,omega,my=batchTrain(Y0,C,20,95,10)

# Treningsalgoritme

In [35]:


def trainingSpiral(I,K,tol):#I er antall bilder, K er antall lag og tol er toleranse i prosent(heltall)
    starttidspunkt=time.time()
    d=2 #Antall bildepunkter 
    tau=0.1 #Definerer tau
    omega,my,W,b = makeOmegaMyWb(K,d,I)
    Y0,C=get_data_spiral_2d(I) #henter inn data Y og fasit C. Y er en liste på formen[[x1,x2,x3,...],[y1,y2,y3,...]] Altså bilder i kolonner
    C=formatC(C)
    dJdWNy = np.zeros((K, d,d)) #Kxdxd
    dJdbNy = np.zeros((K, d, I)) #KxdxI matrise
    iter=0
    maxiter=1000 #Øvre grense
    accuracy=0 #I prosent
    onesII=np.ones((I,I))
    zerosK1dI=np.zeros((K+1,d,I))
    print("Initialisering ferdig")
    acchistory=[]
    while(iter<maxiter and accuracy<tol):
        Y=Yfinder(W,b,Y0)
        Z=prosjisering(Y[-1],omega,my)
        W,b,omega,my=optimeringsfunk(W,b,omega,my,Y,C,tau, dJdWNy, dJdbNy,onesII, zerosK1dI)
        iter+=1
        accuracy=answercheck(Z,C)
        acchistory.append(accuracy)
        if iter%10==0:
            print(iter,"av maxiter",maxiter,"Foreløpig accuracy: ",acchistory[-1],"%")
            if acchistory[-1]<=50.0 and acchistory[-2]<=50.0: #menes det acchistory[-1]<=50.0 and acchistory[-2]<=50.0?
                iter=0
                W=makeW(K,d)
                b=makeb(K,d,I)
                omega=np.ones(d)
                my=0.1
                acchistory=[]
                Y=Yfinder(W,b,Y0)
                print("Resetter")
    sluttidspunkt=time.time()
    print("Kjøretid:",sluttidspunkt-starttidspunkt)
    return W,b,omega,my

def newtrainingspiral(I,K,tol):
    Y0,C=get_data_spiral_2d(I)
    C=formatC(C)
    return generalTrain(Y0,C,K,tol)

def batchspiral(I,K,tol,batches):
    Y0,C=get_data_spiral_2d(I)
    C=formatC(C)
    return batchTrain(Y0,C,K,tol,batches)




# Optimalisere Tau

In [10]:
def trainingSpiralPlot(I,K,versjon):
    #I=Antall bilder
    #K=Antall lag
    d=2 #Antall bildepunkter 
    omega= np.ones(d)#definerer omega np.random.uniform(0, 1, datapkt)
    my=0.1 #definerer my
    W=makeW(K,d) #Lager vekter med tilfeldige verdier
    b=makeb(K,d,I) #Lager bias med tilfeldige verdier
    Y0,C=get_data_spiral_2d(I) #henter inn data Y og fasit C. Y er en liste på formen[[x1,x2,x3,...],[y1,y2,y3,...]] Altså bilder i kolonner
    C=formatC(C)
    iter=0
    maxiter=300 #Øvre grense
    accuracy=0
    tol=97 #I prosent
    if versjon==1:
        tau=0.1 #Definerer tau
        while(iter<maxiter and accuracy<tol):
            if(accuracy>51):
                tau = 0.07
          Y=Yfinder(W,b,Y0)
          Z=prosjisering(Y[-1],omega,my)
          W,b,omega,my=optimeringsfunk(W,b,omega,my,Y,C,tau)
          iter+=1
          accuracy=answercheck(Z,C)
          if iter%10==0:
            print(iter,"av maxiter",maxiter,"Foreløpig accuracy: ",accuracy,"%")
    elif versjon==2:
      tau=0.1 #Definerer tau
      while(iter<maxiter and accuracy<tol):
          Y=Yfinder(W,b,Y0)
          Z=prosjisering(Y[-1],omega,my)
          W,b,omega,my=optimeringsfunk(W,b,omega,my,Y,C,tau)
          iter+=1
          accuracy=answercheck(Z,C)
          if iter%10==0:
            print(iter,"av maxiter",maxiter,"Foreløpig accuracy: ",accuracy,"%")
    elif versjon==3:
      tau=0.01 #Definerer tau
      while(iter<maxiter and accuracy<tol):
          if(accuracy>51):
              tau = 0.07
          Y=Yfinder(W,b,Y0)
          Z=prosjisering(Y[-1],omega,my)
          W,b,omega,my=optimeringsfunk(W,b,omega,my,Y,C,tau)
          iter+=1
          accuracy=answercheck(Z,C)
          if iter%10==0:
            print(iter,"av maxiter",maxiter,"Foreløpig accuracy: ",accuracy,"%")
    return W,b,omega,my,Z,C,iter+1

def gj(liste):
    return sum(liste)/len(liste)

def tesTingTing():
    startTid=time.time()
    CheckList1=[]; CheckList2=[]; CheckList3=[]
    iterList1=[];iterList2=[];iterList3=[]
    for i in range(5):
        W,b,omega,my,Z,C,iter=trainingSpiralPlot(1000,30,1)
        iterList1.append(iter)
        CheckList1.append(answercheck(Z,C))
        W,b,omega,my,Z,C,iter=trainingSpiralPlot(1000,30,2)
        iterList2.append(iter)
        CheckList2.append(answercheck(Z,C))
        W,b,omega,my,Z,C,iter=trainingSpiralPlot(1000,30,3)
        iterList3.append(iter)
        CheckList3.append(answercheck(Z,C))

    print("\n\n\nScenario: Tau lik 0.1 alltid\n\n","Checklist:\n",gj(CheckList1),"\niterList\n",gj(iterList1))
    print("Scenario: Tau lik 0.1, så 0.07 etter 0.51 riktige\n\n","Checklist:\n",gj(CheckList2),"\niterList\n",gj(iterList2))
    print("Scenario: Tau lik 0.01, så 0.07 etter 0.51 riktige\n\n","Checklist:\n",gj(CheckList3),"\niterList\n",gj(iterList3))

    sluttTid=time.time()
    print("Tid",sluttTid-startTid)



IndentationError: unindent does not match any outer indentation level (<ipython-input-10-fcca27bbecdb>, line 20)

# Oppgave 4.1 - Spiral


In [43]:

def getredbluepoints(I,K,tol,batches):
    W,b,omega,my=batchspiral(I,K,tol,batches) #Initialiserer og optimaliserer W,b,omega,my batchvis
    b=finalB(b,I) #Gir b riktig format for å fungere på alle bilder samtidig
    spiralpoints, fasit = get_data_spiral_2d(I) #Henter data fra oppgaven
    fasit=formatC(fasit) #Gjør fasiten(C) leselig
    spirallist=Yfinder(W,b,spiralpoints) #Finner Y fra optimaliserte W,b
    Z=prosjisering(spirallist[-1],omega,my) #Finner Z, som vil være Ix1 med tall i intervallet [0,1]
    spirallist=np.array(spirallist)
    return spiralpoints,Z,fasit,spirallist


startTid = time.time()

sp,Z,C,Ylist=getredbluepoints(10000,20,98,50)
#Z=zconvert(Z) #Gjør nada.##Dette er fordi answercheck allerede runder av verdiene i Z for oss

plot_progression(Ylist,C)
print("Riktig sjekk:",answercheck(Z,C)) #Viser forskjellig fra siste answercheck i batchTrain. Selv med kun 1 batch.

sluttTid = time.time()
print("kjøretid: ", sluttTid-startTid)


Initsialisering ferdig...
avvik: 25.000051527137593
47.5
avvik: 24.999579738907034
47.5
avvik: 24.99910813640835
62.0
avvik: 24.998636703134384
52.5
avvik: 24.99816537631569
52.5
avvik: 24.99769404257453
57.5
avvik: 24.997222603055196
60.0
avvik: 24.996751011658954
63.0
avvik: 24.99627925197533
64.0
avvik: 24.99580731705853
67.0
Iterasjon: 10 føreløpig treffsikkerhet: 67.0 %
avvik: 24.995335202287293
67.5
avvik: 24.994862903191525
67.0
avvik: 24.994390414714392
66.5
avvik: 24.993917730972246
65.5
avvik: 24.993444845247588
65.5
avvik: 24.992971750093147
65.5
avvik: 24.992498437471834
66.0
avvik: 24.992024898890907
65.5
avvik: 24.991551125514082
65.5
avvik: 24.991077108248458
65.5
Iterasjon: 20 føreløpig treffsikkerhet: 65.5 %
avvik: 24.990602837809433
64.5
avvik: 24.990128304768778
64.5
avvik: 24.989653499590087
64.5
avvik: 24.989178412655598
64.5
avvik: 24.988703034286786
64.5
avvik: 24.98822735476078
64.5
avvik: 24.9877513643239
64.5
avvik: 24.987275053202996
64.5
avvik: 24.9867984116

avvik: 24.856758998262812
64.5
avvik: 24.856055615701138
64.5
avvik: 24.85535097336601
64.5
avvik: 24.8546450721757
64.5
avvik: 24.85393791309598
64.5
Iterasjon: 260 føreløpig treffsikkerhet: 64.5 %
avvik: 24.853229497140404
64.5
avvik: 24.852519825370948
64.5
avvik: 24.85180889889832
64.5
avvik: 24.851096718882605
64.5
avvik: 24.85038328653391
64.5
avvik: 24.84966860311298
64.5
avvik: 24.848952669931894
64.5
avvik: 24.848235488355105
64.5
avvik: 24.84751705980029
64.5
avvik: 24.84679738573926
64.5
Iterasjon: 270 føreløpig treffsikkerhet: 64.5 %
avvik: 24.84607646769955
64.5
avvik: 24.84535430726546
64.5
avvik: 24.844630906079846
64.5
avvik: 24.84390626584603
64.5
avvik: 24.843180388329735
64.5
avvik: 24.842453275361773
64.5
avvik: 24.841724928840993
64.5
avvik: 24.84099535073758
64.5
avvik: 24.840264543097366
64.5
avvik: 24.839532508046688
64.5
Iterasjon: 280 føreløpig treffsikkerhet: 64.5 %
avvik: 24.838799247798296
64.5
avvik: 24.83806476465862
64.5
avvik: 24.83732906103652
64.5
avv

avvik: 24.65036665242015
65.0
avvik: 24.649483818421384
65.0
avvik: 24.648600335616027
65.0
avvik: 24.64771620405407
65.0
avvik: 24.646831424044713
65.0
avvik: 24.64594599635828
65.0
avvik: 24.645059922445427
65.0
avvik: 24.64417320466401
65.0
avvik: 24.643285846503463
65.0
avvik: 24.642397852799437
65.0
Iterasjon: 520 føreløpig treffsikkerhet: 65.0 %
avvik: 24.64150922993194
65.0
avvik: 24.640619986003102
65.0
avvik: 24.639730130990667
65.0
avvik: 24.63883967687547
65.0
avvik: 24.637948637739512
65.0
avvik: 24.637057029832853
65.0
avvik: 24.636164871605867
65.0
avvik: 24.63527218370331
65.0
avvik: 24.63437898891707
65.0
avvik: 24.63348531209288
65.0
Iterasjon: 530 føreløpig treffsikkerhet: 65.0 %
avvik: 24.632591179989
65.0
avvik: 24.631696621084437
65.0
avvik: 24.630801665336172
65.0
avvik: 24.62990634388805
65.0
avvik: 24.629010688735427
65.0
avvik: 24.628114732354028
65.0
avvik: 24.627218507303624
65.0
avvik: 24.626322045820825
65.0
avvik: 24.625425379417035
65.0
avvik: 24.62452853

avvik: 24.421646712137427
63.0
avvik: 24.420820901830655
63.0
avvik: 24.4199956103106
63.0
avvik: 24.41917083895502
63.0
avvik: 24.41834658908545
63.0
avvik: 24.417522861968873
63.0
avvik: 24.416699658819386
63.0
avvik: 24.415876980800633
63.0
Iterasjon: 780 føreløpig treffsikkerhet: 63.0 %
avvik: 24.415054829027966
63.0
avvik: 24.414233204571335
63.0
avvik: 24.41341210845773
63.0
avvik: 24.41259154167397
63.0
avvik: 24.41177150516927
63.0
avvik: 24.410951999857698
63.0
avvik: 24.41013302662024
63.0
avvik: 24.409314586306674
63.0
avvik: 24.408496679736768
63.0
avvik: 24.407679307701265
63.0
Iterasjon: 790 føreløpig treffsikkerhet: 63.0 %
avvik: 24.406862470962086
63.0
avvik: 24.406046170251816
63.0
avvik: 24.405230406273027
63.0
avvik: 24.404415179696475
63.0
avvik: 24.403600491158976
63.0
avvik: 24.40278634126093
63.0
avvik: 24.40197273056321
63.0
avvik: 24.401159659583815
63.0
avvik: 24.400347128794554
63.0
avvik: 24.399535138617445
63.0
Iterasjon: 800 føreløpig treffsikkerhet: 63.0 

55.0
Avvik:  24.61528311335249
54.5
Avvik:  24.61464908020589
54.5
Avvik:  24.614015098102506
54.5
Avvik:  24.613381168779416
54.0
Avvik:  24.61274729380204
54.0
Avvik:  24.61211347456819
54.0
Avvik:  24.61147971231261
54.0
Avvik:  24.610846008111725
54.0
Avvik:  24.61021236288873
54.0
Avvik:  24.609578777419124
Iterasjon: 30 føreløpig treffsikkerhet: 54.0 %
54.0
Avvik:  24.608945252336216
54.0
Avvik:  24.60831178813704
54.0
Avvik:  24.60767838518809
54.0
Avvik:  24.6070450437314
54.0
Avvik:  24.606411763890428
54.0
Avvik:  24.605778545676152
54.0
Avvik:  24.605145388992973
54.0
Avvik:  24.60451229364452
54.0
Avvik:  24.6038792593396
54.0
Avvik:  24.603246285697622
Iterasjon: 40 føreløpig treffsikkerhet: 54.0 %
54.0
Avvik:  24.602613372254307
54.0
Avvik:  24.60198051846707
54.0
Avvik:  24.60134772371994
54.0
Avvik:  24.600714987328878
54.0
Avvik:  24.600082308546522
54.0
Avvik:  24.599449686566683
54.0
Avvik:  24.598817120529006
54.0
Avvik:  24.598184609522903
53.5
Avvik:  24.597552152

...

50.0
Avvik:  23.885573731744685
50.0
Avvik:  23.885158034166313
Iterasjon: 220 føreløpig treffsikkerhet: 50.0 %
50.0
Avvik:  23.884742592108797
50.0
Avvik:  23.884327404925436
50.0
Avvik:  23.88391247196936
50.0
Avvik:  23.88349779259323
50.0
Avvik:  23.88308336614917
50.0
Avvik:  23.88266919198852
50.0
Avvik:  23.88225526946176
50.0
Avvik:  23.881841597918395
50.0
Avvik:  23.88142817670667
50.0
Avvik:  23.881015005173612
Iterasjon: 230 føreløpig treffsikkerhet: 50.0 %
49.5
Avvik:  23.88060208266478
49.5
Avvik:  23.880189408524142
49.5
Avvik:  23.879776982094047
49.5
Avvik:  23.879364802714896
49.5
Avvik:  23.878952869725282
49.5
Avvik:  23.878541182461674
49.5
Avvik:  23.8781297402584
49.5
Avvik:  23.877718542447465
49.5
Avvik:  23.877307588358505
49.5
Avvik:  23.87689687731865
Iterasjon: 240 føreløpig treffsikkerhet: 49.5 %
49.5
Avvik:  23.876486408652454
49.0
Avvik:  23.876076181681693
49.0
Avvik:  23.875666195725405
49.0
Avvik:  23.875256450099727
49.0
Avvik:  23.874846944117735
49

49.0
Avvik:  23.796187699344134
49.0
Avvik:  23.795807900202597
49.0
Avvik:  23.795428157760227
49.0
Avvik:  23.795048471413676
49.0
Avvik:  23.79466884056219
Iterasjon: 450 føreløpig treffsikkerhet: 49.0 %
49.0
Avvik:  23.79428926460743
49.0
Avvik:  23.793909742953375
49.0
Avvik:  23.793530275006106
49.0
Avvik:  23.793150860173803
49.0
Avvik:  23.79277149786657
49.0
Avvik:  23.792392187496258
49.0
Avvik:  23.792012928476414
49.0
Avvik:  23.791633720222123
49.0
Avvik:  23.791254562149923
49.0
Avvik:  23.790875453677632
Iterasjon: 460 føreløpig treffsikkerhet: 49.0 %
49.0
Avvik:  23.79049639422436
49.0
Avvik:  23.790117383210312
49.0
Avvik:  23.789738420056626
49.0
Avvik:  23.789359504185448
49.0
Avvik:  23.78898063501967
49.0
Avvik:  23.78860181198291
49.0
Avvik:  23.78822303449941
49.0
Avvik:  23.787844301993935
49.0
Avvik:  23.787465613891687
49.0
Avvik:  23.78708696961822
Iterasjon: 470 føreløpig treffsikkerhet: 49.0 %
49.0
Avvik:  23.78670836859935
49.0
Avvik:  23.786329810261144
4

48.5
Avvik:  23.695099607294853
48.5
Avvik:  23.69471813782268
48.5
Avvik:  23.694336651990128
48.5
Avvik:  23.69395515042095
48.5
Avvik:  23.693573633750475
48.5
Avvik:  23.693192102625545
48.5
Avvik:  23.6928105577046
48.5
Avvik:  23.692428999657807
Iterasjon: 720 føreløpig treffsikkerhet: 48.5 %
48.5
Avvik:  23.692047429167022
48.5
Avvik:  23.69166584692599
48.5
Avvik:  23.691284253640355
48.5
Avvik:  23.690902650027645
48.5
Avvik:  23.69052103681749
48.5
Avvik:  23.69013941475154
48.5
Avvik:  23.689757784583588
48.5
Avvik:  23.689376147079642
48.5
Avvik:  23.688994503017984
48.5
Avvik:  23.688612853189138
Iterasjon: 730 føreløpig treffsikkerhet: 48.5 %
48.5
Avvik:  23.68823119839606
48.5
Avvik:  23.68784953945403
48.5
Avvik:  23.687467877190837
48.5
Avvik:  23.687086212446765
48.5
Avvik:  23.68670454607461
48.5
Avvik:  23.686322878939766
48.5
Avvik:  23.685941211920188
48.5
Avvik:  23.68555954590657
48.5
Avvik:  23.68517788180221
48.5
Avvik:  23.68479622052319
Iterasjon: 740 førelø

49.0
Avvik:  23.600987080583625
49.0
Avvik:  23.60064268036553
49.0
Avvik:  23.600298575907217
49.0
Avvik:  23.59995476764029
Iterasjon: 970 føreløpig treffsikkerhet: 49.0 %
49.0
Avvik:  23.59961125598655
49.0
Avvik:  23.59926804135802
49.0
Avvik:  23.598925124157205
49.0
Avvik:  23.598582504777013
49.0
Avvik:  23.598240183600762
49.0
Avvik:  23.59789816100241
49.0
Avvik:  23.59755643734647
49.0
Avvik:  23.59721501298819
49.0
Avvik:  23.596873888273453
49.0
Avvik:  23.596533063539155
Iterasjon: 980 føreløpig treffsikkerhet: 49.0 %
49.0
Avvik:  23.596192539112906
49.0
Avvik:  23.59585231531337
49.0
Avvik:  23.59551239245024
49.0
Avvik:  23.5951727708243
49.0
Avvik:  23.594833450727492
49.0
Avvik:  23.594494432443156
49.0
Avvik:  23.594155716245865
49.0
Avvik:  23.593817302401682
49.0
Avvik:  23.593479191168203
49.0
Avvik:  23.59314138279457
Iterasjon: 990 føreløpig treffsikkerhet: 49.0 %
49.0
Avvik:  23.59280387752171
49.0
Avvik:  23.592466675582287
49.0
Avvik:  23.592129777200853
49.0


KeyboardInterrupt: 

# Oppgave 4.2 - MNIST

In [47]:
from loader import get_dataset

def mnistTrain(K):
    I=5
    d=28*28 #Bildedimensjon
    tau=0.1
    Y0,C=get_dataset("training")
    Y0=Y0.transpose()[:I].transpose()
    print("Y0:\n",Y0)
    print("\n\n\nC:\n",C)

    omega,my,W,b = makeOmegaMyWb(K,d,I)
    
    Kdd0 = np.zeros((K, d,d)) #Kxdxd med 0 i hvert element
    KdI0 = np.zeros((K, d, I)) #KxdxI med 0 i hvert element
    II1=np.ones((I,I)) #IxI med 1 i hvert element
    iter=0
    maxiter=1000

    C=formatC(C); C=C[:I]
    print("\n\nC:\n",C)
    iter=0
    maxiter=1000 #Øvre grense
    accuracy=0
    tol=98 #I prosent
    print("Initsialisering ferdig")
    acchistory=[]
    print(C)
    while(iter<maxiter and accuracy<tol):
        Y=Yfinder(W,b,Y0)
        Z=prosjisering(Y[-1],omega,my)
        W,b,omega,my=optimeringsfunk(W,b,omega,my,Y,C,tau, Kdd0, KdI0,II1)
        iter+=1
        accuracy=answercheck(Z,C)
        acchistory.append(accuracy)
        if iter%10==0:
          print(iter,"av maxiter",maxiter,"Foreløpig accuracy: ",accuracy,"%")
          '''if acchistory[-1] and acchistory[-2]<=50.0:
              iter=0
              W=makeW(K,d)
              b=makeb(K,d,I)
              omega=np.ones(d)
              my=0.1
              acchistory=[]
              Y=Yfinder(W,b,Y0)
              print("Resetter")'''
    print(acchistory)
    return W,b,omega,my,Z,C

def mnistbatchtrain(K,tol):
    Y0,fasitt=get_dataset("training",1,8)
    Y0=Y0 #formatering
    batches=len(Y0[0])//128
    print(batches)
    fasitt=formatC(fasitt)
    W,b,omega,my = batchTrain(Y0,fasitt,K,tol,batches)
    b=finalB(b,len(Y0[0]))
    return W,b,omega,my

W,b,omega,my = mnistbatchtrain(20,95)
Y0,fasit2=get_dataset("testing",1,8)
print(len(Y0),len(Y0[0]))

fasit2=formatC(fasit2)
b=finalB(b,len(Y0[0]))
Ylist=Yfinder(W,b,Y0)
Z=prosjisering(Ylist[-1],omega,my)
print("Endelig test:",answercheck(Z,fasit2))



98
Initsialisering ferdig...
avvik: 16.009980662955805
61.71875
avvik: 8.77830394506498
89.84375
avvik: 5.338788168412571
92.1875
avvik: 3.9037751021076943
92.96875
avvik: 3.2093475812782093
93.75
avvik: 2.808825553347275
93.75
avvik: 2.4885808354648504
94.53125
avvik: 2.1850434115356103
94.53125
avvik: 1.892516499828892
95.3125
avvik: 1.6244948585246197
96.09375
Iterasjon: 10 føreløpig treffsikkerhet: 96.09375 %
Kjøretid: 13.481684923171997 
Treffsikkerthet: 96.09375 %
96.875
Avvik:  1.985719085148622
Kjøretid: 1.3783459663391113 
Treffsikkerthet: 96.875 %
Breaker fordi god nok toleranse
Accuracy fra siste batchen: 96.875
784 2109
Endelig test: 95.54291133238502


In [ ]:
#Nablaoperator
def nabJGammel(W, b , omega, my,Y,C): #Kanskje fette unødvendig å bruke U her, for så å unnpacke denne shaiten
    YK=Y[-1]
    Z = prosjisering(YK, omega, my)
    dJdWNy = [0 for i in range(len(W))]
    dJdbNy = [0 for i in range(len(W))]
    dJomega = dJdomega(YK, omega, my, Z, C) #???
    dJmy = dJdmy(YK, omega, my, Z, C) #???
    P = P_finder(Y,W,b,omega,my,Z,C)
    for i in range(0, len(W)):
        dJdWNy[i]=dJdW(P[i+1], W[i], Y[i], b[i])
        dJdbNy[i]=dJdb(P[i+1],W[i],Y[i],b[i])
    dJdWNy=np.array(dJdWNy)
    dJdbNy=np.array(dJdbNy)
    return dJdWNy,dJdbNy,dJomega,dJmy